In [2]:
import pandas as pd
import re
import ast
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from tabulate import tabulate
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv('final_dataset.csv')

In [4]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

In [5]:
df.columns

Index(['name', 'recipe_id', 'minutes', 'n_steps', 'steps', 'ingredients',
       'n_ingredients', 'cuisine', 'time_to_make', 'minutes_category',
       'steps_category', 'calories', 'total_fat', 'sugar', 'sodium', 'protein',
       'saturated_fat', 'carbohydrates', 'ingredients_combined', 'rating_1',
       'rating_2', 'rating_3', 'rating_4', 'rating_5', 'num_rating',
       'cuisine_indian', 'cuisine_chinese', 'cuisine_russian',
       'cuisine_middle_eastern', 'cuisine_oceanian', 'cuisine_canadian',
       'cuisine_caribbean_latin _american', 'cuisine_african', 'cuisine_asian',
       'cuisine_european', 'cuisine_american', 'cuisine_australian',
       'diet_type_gluten_free', 'diet_type_diary_free',
       'diet_type_special_nutrient_focus', 'diet_type_low_carb',
       'diet_type_low_protein', 'diet_type_diabetic_friendly',
       'diet_type_kosher', 'diet_type_high_protein', 'diet_type_high_fiber',
       'diet_type_vegetarian', 'diet_type_healthy', 'diet_type_low_calorie',
      

In [6]:
def process_ingredients(ingredients_str):
    ingredients_str = ingredients_str.lower()
    l = ingredients_str.split()
    l = [lemmatizer.lemmatize(item) for item in l]
    l = [item for item in l if item not in stop_words]
    return ' '.join(l)

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF feature extractor
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df['ingredients_combined'])

In [8]:
diet_type_dict = {
        'Gluten Free' : 'diet_type_gluten_free', 
        'Diary Free': 'diet_type_diary_free',
        'Special Nutrient Food' : 'diet_type_special_nutrient_focus', 
        'Low Carb' : 'diet_type_low_carb',
        'Low Protein' : 'diet_type_low_protein', 
        'Diabetic Friendly' : 'diet_type_diabetic_friendly',
        'Koshner' : 'diet_type_kosher', 
        'High Protein' : 'diet_type_high_protein', 
        'High Fiber' : 'diet_type_high_fiber',
        'Vegetarian' : 'diet_type_vegetarian', 
        'Healthy': 'diet_type_healthy', 
        'Low Calorie' : 'diet_type_low_calorie',
        'Low Fat' : 'diet_type_low_fat', 
        'Low Saturated Fat' : 'diet_type_low_saturated_fat',
        'Low Sodium' : 'diet_type_low_sodium', 
        'Low Cholesterol' : 'diet_type_low_cholesterol', 
        'Vegan' : 'diet_type_vegan',
        'Calcium' : 'diet_type_high_calcium', 
        'Lactose Free' : 'diet_type_lactose_free',
        'Non Vegetarian' : 'diet_type_non_vegetarian' }

cuisine_dict = {
    'Australian':'cuisine_australian', 
    'Canadian':'cuisine_canadian',
    'European':'cuisine_european', 
    'Chinese' :'cuisine_chinese',
    'Middle Eastern' : 'cuisine_middle_eastern',
    'American':'cuisine_american', 
    'African':'cuisine_african', 
    'Indian':'cuisine_indian',
    'Russian' : 'cuisine_russian', 
    'Asian' : 'cuisine_asian', 
    'Oceanian' : 'cuisine_oceanian',
    'Caribbean/Latin American' : 'cuisine_caribbean_latin _american'
}

In [9]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

def get_recipes(input_str, time_to_make, diet_type, cuisine):
    
    if input_str != "":
        input_str = process_ingredients(input_str)
        input_tfidf = tfidf.transform([input_str])
        cosine_similarities = cosine_similarity(input_tfidf, tfidf_matrix)
        all_recipe_indices_sorted = cosine_similarities.argsort()[0][-30:][::-1]
        df_new = df.iloc[all_recipe_indices_sorted]  
        
    elif input_str == "":
        df_new = df.sort_values(by = ['num_rating','n_ingredients'], ascending = [False, True])
    
    ## filtering the results on time to make 
    if time_to_make != "No preference":
        df_new = df_new[df_new['time_to_make'] == time_to_make]
    
    ## filtering the results on diet type
    if diet_type != "No preference":
        col_name1 = diet_type_dict[diet_type]
        df_new = df_new[df_new[col_name1] == 1]
    
    ## filtering the results on cuisine
    if cuisine != "No preference":
        col_name2 = cuisine_dict[cuisine]
        df_new = df_new[df_new[col_name2] == 1]
        
    df_new = df_new[['name', 'recipe_id', 'minutes', 'n_steps', 'steps', 'ingredients',
        'n_ingredients', 'cuisine', 'time_to_make', 'minutes_category',
       'steps_category', 'calories', 'total_fat', 'sugar', 'sodium', 'protein',
       'saturated_fat', 'carbohydrates', 'ingredients_combined', 'rating_1',
       'rating_2', 'rating_3', 'rating_4', 'rating_5', 'num_rating']]
    
    top_recipes = df_new.sort_values(by = ['n_ingredients','num_rating','minutes'], ascending = [True, False, True])
    
    if top_recipes.shape[0] == 0:
        return "Sorry! No recipes found, try with only ingredients"
    
    return top_recipes[:5]

    

In [27]:
get_recipes("chicken spinach","No preference","No preference","No preference")
df_output = get_recipes("chicken spinach","No preference","No preference","No preference")
df_output


,name,recipe_id,minutes,n_steps,steps,ingredients,n_ingredients,cuisine,time_to_make,minutes_category,...,protein,saturated_fat,carbohydrates,ingredients_combined,rating_1,rating_2,rating_3,rating_4,rating_5,num_rating
29641,comfort soup spinach meatballs,57424,40,8,"['heat chicken broth to boiling', 'meanwhile ,...","['chicken broth', 'beef', 'parmesan cheese', '...",4,"['European', 'American']",Within 1 hour,medium,...,62,40,1,chicken broth beef parmesan cheese spinach,1,1,4,10,22,38
22737,chicken breasts stuffed with feta and spinach,2364,0,8,"['preheat oven to 350 degrees , wash and dry c...","['chicken', 'feta cheese', 'spinach', 'lemon j...",4,[],Within 15 minutes,less,...,40,22,1,chicken feta cheese spinach lemon juice,0,0,1,5,17,23
101695,spinach burgers,263874,25,3,"['mix ground chicken , spinach , breadcrumbs a...","['chicken', 'spinach', 'breadcrumbs', 'garlic ...",4,[],Within 30 minutes,less,...,46,13,2,chicken spinach breadcrumb garlic clove,0,0,0,3,16,19
423,10 minute creamed spinach,94547,10,5,['add the soup mix and milk to a large saucepa...,"['cream of spinach soup mix', 'milk', 'chopped...",4,[],Within 15 minutes,less,...,15,9,3,cream spinach soup mix milk chopped spinach nu...,1,1,0,0,10,12
65377,low calorie spinach mushroom wedding soup,237726,10,6,"['bring chicken broth to a simmer', 'add parme...","['chicken broth', 'parmesan cheese', 'spinach ...",4,[],Within 15 minutes,less,...,32,10,2,chicken broth parmesan cheese spinach leaf mus...,0,0,1,2,5,8


In [11]:
get_recipes("chicken pasta","No preference","No preference","No preference")

,name,recipe_id,minutes,n_steps,steps,ingredients,n_ingredients,cuisine,time_to_make,minutes_category,...,protein,saturated_fat,carbohydrates,ingredients_combined,rating_1,rating_2,rating_3,rating_4,rating_5,num_rating
79651,pasta with bell pepper onion sauce,360544,20,6,['cook pasta in salt water according to packag...,"['pasta', 'bell pepper', 'onion', 'cream']",4,[],Within 30 minutes,less,...,41,107,37,pasta bell pepper onion cream,0,1,0,5,6,12
79865,pastina gasiov,88705,20,7,"['cook the pasta as instructed on the box', 'd...","['pasta', 'parmesan cheese', 'eggs', 'milk']",4,['European'],Within 30 minutes,less,...,45,51,28,pasta parmesan cheese egg milk,0,0,0,1,3,4
39658,easy chicken parmesan,104261,50,8,"['preheat oven to 375 degrees', 'pour sauce in...","['pasta sauce', 'parmesan cheese', 'chicken', ...",4,[],Within 1 hour,medium,...,75,26,4,pasta sauce parmesan cheese chicken mozzarella...,0,0,0,0,4,4
22594,chicken baked ziti,33217,50,8,"['preheat oven to 350f', 'boil pasta until coo...","['ziti pasta', 'pasta sauce', 'mozzarella chee...",4,[],Within 1 hour,medium,...,60,33,15,ziti pasta pasta sauce mozzarella cheese chicken,0,0,0,0,3,3
98189,solo honey lime chicken and pasta,68479,13,3,['cook the pasta in 2 quarts of salted water f...,"['chicken', 'pasta', 'lime', 'honey']",4,[],Within 15 minutes,less,...,108,35,31,chicken pasta lime honey,0,0,0,2,0,2


In [12]:
get_recipes("","No preference","No preference","Asian")

,name,recipe_id,minutes,n_steps,steps,ingredients,n_ingredients,cuisine,time_to_make,minutes_category,...,protein,saturated_fat,carbohydrates,ingredients_combined,rating_1,rating_2,rating_3,rating_4,rating_5,num_rating
70516,minado s perfect sushi rice,119373,90,15,['ignore the directions on the bag that the ri...,"['sushi rice', 'rice vinegar', 'mirin', 'kelp']",4,['Asian'],Within 2 hours,more,...,12,0,28,sushi rice rice vinegar mirin kelp,4,1,1,7,113,126
108906,the best sweet and sour sauce,76003,10,3,"['combine all ingredients in a sauce pan', 'co...","['pineapple juice', 'vinegar', 'soy sauce', 'c...",4,['Asian'],Within 15 minutes,less,...,3,0,36,pineapple juice vinegar soy sauce cornstarch,1,1,5,15,91,113
60069,juicy cornish game hens,52624,60,9,"['preheat oven to 350f', 'melt butter and stir...","['honey', 'chili powder', 'orange juice', 'cor...",4,"['Asian', 'American']",Within 1 hour,more,...,49,41,4,honey chili powder orange juice cornish hen,0,2,6,13,59,80
76715,onion lentils and rice,30698,40,11,"['heat olive oil in skillet , saute the onions...","['onions', 'lentils', 'rice', 'cumin']",4,"['Asian', 'Middle Eastern']",Within 1 hour,medium,...,23,10,25,onion lentil rice cumin,1,2,1,8,39,51
87105,quick chinese fried rice,22910,20,10,"['boil 1 cup water and the rice , stir and let...","['minute rice', 'egg', 'onion', 'soy sauce']",4,['Asian'],Within 30 minutes,less,...,11,29,9,minute rice egg onion soy sauce,0,0,2,8,31,41


In [13]:
get_recipes("","No preference","Healthy","Asian")

,name,recipe_id,minutes,n_steps,steps,ingredients,n_ingredients,cuisine,time_to_make,minutes_category,...,protein,saturated_fat,carbohydrates,ingredients_combined,rating_1,rating_2,rating_3,rating_4,rating_5,num_rating
70516,minado s perfect sushi rice,119373,90,15,['ignore the directions on the bag that the ri...,"['sushi rice', 'rice vinegar', 'mirin', 'kelp']",4,['Asian'],Within 2 hours,more,...,12,0,28,sushi rice rice vinegar mirin kelp,4,1,1,7,113,126
108906,the best sweet and sour sauce,76003,10,3,"['combine all ingredients in a sauce pan', 'co...","['pineapple juice', 'vinegar', 'soy sauce', 'c...",4,['Asian'],Within 15 minutes,less,...,3,0,36,pineapple juice vinegar soy sauce cornstarch,1,1,5,15,91,113
76715,onion lentils and rice,30698,40,11,"['heat olive oil in skillet , saute the onions...","['onions', 'lentils', 'rice', 'cumin']",4,"['Asian', 'Middle Eastern']",Within 1 hour,medium,...,23,10,25,onion lentil rice cumin,1,2,1,8,39,51
64108,lentils garlic and tomatoes,16666,50,8,['heat the oil in a large 2 quart pot over med...,"['garlic', 'tomatoes', 'lentils', 'lemon juice']",4,"['Asian', 'Indian']",Within 1 hour,medium,...,26,3,10,garlic tomato lentil lemon juice,0,0,4,5,27,36
56951,hummus without tahini,30084,7,4,['put all ingredients into a food processor an...,"['chickpeas', 'lemon', 'garlic clove', 'red pe...",4,"['Asian', 'Middle Eastern']",Within 15 minutes,less,...,14,13,11,chickpea lemon garlic clove red pepper flake,0,1,1,3,29,34


In [14]:
get_recipes("","Within 1 hour","Healthy","Asian")

,name,recipe_id,minutes,n_steps,steps,ingredients,n_ingredients,cuisine,time_to_make,minutes_category,...,protein,saturated_fat,carbohydrates,ingredients_combined,rating_1,rating_2,rating_3,rating_4,rating_5,num_rating
76715,onion lentils and rice,30698,40,11,"['heat olive oil in skillet , saute the onions...","['onions', 'lentils', 'rice', 'cumin']",4,"['Asian', 'Middle Eastern']",Within 1 hour,medium,...,23,10,25,onion lentil rice cumin,1,2,1,8,39,51
64108,lentils garlic and tomatoes,16666,50,8,['heat the oil in a large 2 quart pot over med...,"['garlic', 'tomatoes', 'lentils', 'lemon juice']",4,"['Asian', 'Indian']",Within 1 hour,medium,...,26,3,10,garlic tomato lentil lemon juice,0,0,4,5,27,36
61892,kushi dango skewered sweet dumplings,120390,55,9,['put rice flour in a bowl and add warm water'...,"['glutinousrice flour', 'soy sauce', 'potato s...",4,['Asian'],Within 1 hour,medium,...,7,0,20,glutinousrice flour soy sauce potato starch stick,1,1,3,5,6,16
87273,quick naan bread machine,254744,40,16,"['if the yogurt is cold , heat it for 15 secon...","['honey', 'yogurt', 'flour', 'yeast']",4,"['Asian', 'Indian']",Within 1 hour,medium,...,14,4,16,honey yogurt flour yeast,0,0,0,3,11,14
20064,champorado chocolate rice pudding,132659,35,5,['cook rice in a medium-size saucepan with wat...,"['sweet rice', 'unsweetened cocoa', 'vanilla e...",4,['Asian'],Within 1 hour,medium,...,10,4,23,sweet rice unsweetened cocoa vanilla extract s...,0,0,1,3,7,11


In [34]:
df_output = get_recipes("","Within 1 hour","Healthy","Asian")

In [43]:
import plotly.express as px

# Assuming df_output is your dataframe
# Renaming columns
df_output = df_output.rename(columns={'rating_1': '1 Star', 'rating_2': '2 Stars', 'rating_3': '3 Stars', 'rating_4': '4 Stars', 'rating_5': '5 Stars'})

# Iterate over each recipe in df_output
for index, row in df_output.iterrows():
    # Create a new dataframe for the current recipe
    recipe_df = pd.DataFrame({'Rating': ['1 Star', '2 Stars', '3 Stars', '4 Stars', '5 Stars'],
                              'Frequency': row[['1 Star', '2 Stars', '3 Stars', '4 Stars', '5 Stars']].values})
    
    # Create the interactive bar chart using Plotly Express for the current recipe
    fig = px.bar(recipe_df, x='Rating', y='Frequency', title=f'Frequency of Ratings for Recipe: {row["name"]}', 
                 labels={'Frequency': 'Frequency', 'Rating': 'Rating'}, color='Rating',
                 color_discrete_map={'1 Star': 'rgb(152,251,152)', '2 Stars': 'rgb(144,238,144)', 
                                     '3 Stars': 'rgb(50,205,50)', '4 Stars': 'rgb(0,170,0)', 
                                     '5 Stars': 'rgb(0,128,0)'})
    
    # Modifying the layout to set the background color to black and remove gridlines
    fig.update_layout(
        plot_bgcolor='black',  # Set plot background color to black
        paper_bgcolor='black',  # Set paper background color to black
        font_color='white',  # Set font color to white
        xaxis=dict(showgrid=False),  # Remove x-axis gridlines
        yaxis=dict(showgrid=False),  # Remove y-axis gridlines
        coloraxis_colorbar=dict(
            title='Frequency',
            tickvals=[0, max(recipe_df['Frequency'])],
            ticktext=['Light', 'Dark']
        )
    )
    
    # Show the chart for the current recipe
    fig.show()






 




In [46]:
import plotly.graph_objects as go

def plot_nutrition(data):
    '''
    Plots a horizontal stacked bar chart showing the nutritional facts of each recipe
    Returns a plotly.graph_objects.Figure
    '''
    # Extracting the recipe name and capitalizing it
    recipe_title = data['name'].capitalize()

    # Extracting the nutrients and their values from the DataFrame
    nutrients = ['calories', 'total_fat', 'sugar', 'sodium', 'protein', 'saturated_fat', 'carbohydrates']
    colors = ['rgb(31, 119, 180)', 'rgb(255, 127, 14)', 'rgb(44, 160, 44)', 'rgb(214, 39, 40)', 'rgb(148, 103, 189)', 'rgb(140, 86, 75)', 'rgb(227, 119, 194)']  # Different colors for each nutrient

    x = nutrients
    y = [data[nutrient] for nutrient in nutrients]  # Convert values to list

    fig = go.Figure()

    for nutrient, color, value in zip(nutrients, colors, y):
        # Capitalize the nutrient name
        if nutrient == 'total_fat':
            nutrient_name = 'Total fat'
        elif nutrient == 'saturated_fat':
            nutrient_name = 'Saturated fat'
        else:
            nutrient_name = nutrient.capitalize()

        fig.add_trace(go.Bar(
            y=[nutrient_name],
            x=[value],
            orientation='h',
            name=nutrient_name,  # Use capitalized nutrient name as legend
            marker_color=color,
            hovertemplate="<br><b>%{y}</b>: %{x:.2f}",
            showlegend=False,
        ))

    fig.update_layout(
        template="plotly_dark",
        margin=dict(l=20, r=20, t=50, b=20),  # Increase top margin to 50
        width=500,
        height=250,  # Increase height to 250
        xaxis=dict(
            showgrid=False,
        ),
        yaxis=dict(
            showgrid=False,
        ),
        title=recipe_title,  # Set the title of the plot to the capitalized recipe title
    )

    return fig

# Iterate over each row in the DataFrame of recipes and display the nutrition plot for each recipe
for index, row in df_output.iterrows():
    fig = plot_nutrition(row)
    fig.show()


